In [1]:
import sys
import numpy as np
import pandas as pd
# For ease of running, make sure requestSheetFunctions.py is in the same folder as this notebook
# Note the functions sheet requires astropy, astroplan, and requests installed on your machine
# These can be pip installed: 
# https://www.astropy.org 
# https://astroplan.readthedocs.io/en/stable/installation.html
# https://pypi.org/project/requests/
import requestSheetFunctions as rsf


# Instructions
All functions are stored in file requestSheetFunctions.py. Make sure this file is in the same folder as this jupyter notebook. 

There are two classes: Requests and Programs. You will first generate a Request object for each target by instantiating a new Request object and then manually entering information about the target as well as desired observational strategy. Options to pull coordinates from SIMBAD are included, be sure the target name is queriable. **See below for two examples.** Once all information is inputed, you will run 2 functions within the Request object: checkForNones() and runChecks(). These will ensure that all formatting is correct. If there are errors, messages will be outputted to you. Fix all errors and rerun these functions before continuing. 

Optional but encouraged to run computeTimeRequest(), isObservable(), and determineFeasibility(). These will calculate total time required to complete request (including overheads), check how many days in the sememster the target is accessible from Keck Observatory, and perform a first order check of the feasibility of the request.

Once all Request objects are created and passing formatting tests, instantiate a Program object and give it a list of request objects. **See below for an example.** Then run three final functions: totalProgramTime(), checkAllRequests(), and writeFile(). These compute stats for your program, check formatting again, and then write out your requests to a csv. Note, the totalProgramTime() function in the Program class will not work if you haven't run computeTimeRequest() function in the Request class for each of your requests.

Finally, email your csv to jblubin@astro.ucla.edu. Also email if any questions. 

# Definitions, please read carefully. 
Reach out to jblubin@astro.ucla.edu with any questions

**exposure_time** -- (int) The requested exposure time for each observation, in seconds. You can determine the ideal exposure time using the KPF Exposure Time Calculator (ETC), found here: https://github.com/California-Planet-Search/KPF-etc. At this time, the nominal and max exposure time must be the same value, as the KPF team is not currently supporting dynamic exposure times dependent on SNR. 

**n_observations_per_visit** -- (int) The number of observations (exposures) for each visit within a single night. For example, for a triple shot put 3 here. To compute overhead, we add (N_observations_per_visit - 1)x45 seconds to your request. This is to account for readout time. The -1 is to acknowledge that after the last observation in a series, readout occurs during slew time and we don't want to double charge.

**n_visits_per_night** -- (int) The number of times to observe (visit) the star within a night. For example, if you want to observe this star once and then again later in the same night separated by X hours, put 2 here. Note, each visit will get n_observations_per_visit worth of exposures. If this value is greater than 1, you will also need to specify an intra_night_cadence, see below. To compute overhead, we add (n_observations_per_visit x 240) seconds to your request. This is to account for average slew and acquisition time between targets, as well as global time sinks, like focus and calibrations.

**intra_night_cadence** -- (float) The time, in units of hours, that you wish to be the minimum separation between visits in the same night. If N_visits_per_night is equal to 1, then this number must be set to 0. This value may be a float to allow for fractional hour separations.

**n_unique_nights** -- (int) The number of unique nights in the semester to observe the star. Each unique night, the star will be observed n_visits_per_night times, with each visit obtaining n_observations_per_visit number of exposures. Note, historically, RV queue instruments (HIRES and now KPF) have been scheduled for community cadence nights on ~80 unique nights. In a given semester, the total number of nights KPF is scheduled is the maximum possible value for n_unique_nights. Once the KPF schedule is announced and programs are allocated time, PI's who have targets with n_unique_nights > unique nights scheduled will be contacted and advised to ammend their request or accept the limiting maximum. 

**inter_night_cadence** -- (int) The time, in days, that you wish to be the minimum separation between observations on unique nights. For example, if you wish to observe this target with at least 10 unique days separation between observations, put 10 here. Note these are calendar days, not KPF scheduled days. If no value is specified, the default value is set to 1 day, which means the target can be observed on every possible KPF scheduled night, until its requested total observations are achieved. If you wish to only get a single observation of a target, set inter_night_cadence equal to 0.

**startDate** -- (str) The calendar day of the semester (UT), with format "YYYY-MM-DD", that you wish to be the first day where your target is allowed to be observed. For example, your target may be accessible from Hawaii on the first day of the A semester (Feb 1), but you wish to wait to observe it until the end of the semester (perhaps to overlap with observations from other facilities), so here you might designate startDate = '2024-05-01'. 

**endDate** -- (str) The calendar day of the semester (UT), with format "YYYY-MM-DD", that you wish to be the last day where your target is allowed to be observed. For example, your target may be accessible from Hawaii all the way until the last day of the A semester (July 31), but you wish to observe it sooner to the beginning of the semester (perhaps to overlap with observations from other facilities), so here you might designate endDate = '2024-03-01'. Note that endDate must be a later date than startDate.

Note: if your observation is an RM or similar, do not use the startDate and endDate fields. Only use the mustDate field (see below).

**mustDate** -- (list of strings) The specific calendar day(s) of the semester (UT), each with format "YYYY-MM-DD", that you must have your target observed (assuming good weather). For example, if a planet reaches quadrature in its orbit on three days of the semester and you wish to hit these dates, specify as a list of strings: ['2024-04-01', '2024-04-12', '2024-04-24']. We cannot gaurentee these kinds of timed observations at this time, but we will do our best to accomodate. Also, if your observation is an RM or similar, please use this field to specify the date.

**notes** -- (str) This is where you can write in any notes to us that you like. Examples include directions in a crowded star field, the component of a visual binary to observe, etc. 




Note: we require the J band magnitude as this is used by the observatory to set the guide camera exposure time. This value will be pulled from Simbad where possible, but not all resolvable targets have J band magnitudes on Simbad. You may need to set the value manually, see examples below.

Note: we require the effective temperature of the star as this is used to determine the CCF mask that is used as part of the RV calculation.

# Example 1
Request to observe a triple shot of a star once per night. Get 50 total unique nights separated by at least 3 days.  

In [2]:
# Initialize a new Request object with the simbad resolvable star name
Example1 = rsf.Request("Sirius")

# query simbad for star info
Example1.get_star_info()

# If your star name is not resolved by simbad, try another name first.
# If still not resolved, then you will need to supply a few coordinates manually. 
# Do so by uncommenting these lines and filling out values manually.
# # Coordinate info
# self.RA = None      # units of degrees, eg. 310.15 
# self.Dec = None     # units of degrees, eg. 46.30
# self.pmRA = None    # units of milliarcseconds per year
# self.pmDec = None   # units of milliarcseconds per year
# self.epoch = None   # usually J2000 or J2015

# # In this case, Sirius does not have a Gaia DR1, DR2, or DR3 name. 
# # So we will have to set a holder name in order to allow the code to work. 
Example1.gaia_name = "No Gaia Name"

# Host star info - manual input
Example1.Teff = 5708
# If SIMBAD doesn't have the J mag (or if target name does not resolve), 
# you must add it here manually from another source like so: 
# Example1.Jmag = 0.0

# Observation info - manual input
Example1.simulcal = True # unless science needs require not taking simultaneous calibrations, please leave True.
Example1.nominal_ExpTime = 10
Example1.max_ExpTime = 10 # For now, max_ExpTime must equal nominal_ExpTime

# Cadence info
# Please review instructions above for info on how to fill out these paramters
Example1.n_observations_per_visit = 3        #int
Example1.n_visits_per_night = 1              #int
Example1.n_unique_nights_per_semester = 50   #int
Example1.intra_night_cadence = 0.            #float - units of hours
Example1.inter_night_cadence = 3             #int   - units of days

# be sure not to include an commas in your note otherwise it messes up the resulting CSV!
Example1.notes = 'This is the brightest star in the sky... cant miss it!'

# Run checks and calculations
Example1.computeTimeRequest()
Example1.runChecks()
if Example1.canContinue:
    print("Passed all checks.")
    Example1.isObservable()
    Example1.determineFeasibility()
else:
    print("Did not pass all checks. Make changes, then re-run.")
    


Checking for parameters still set to None.

Checking for correct data types.

Checking for formatting.

All looks good for this request! 

Passed all checks.
This target is observable for a total of [92] days this semester.
This target rises on day [0] and sets on day [92] of the semester. 

Your program may not be feasible. With your desired inter-night cadence and desired unique night visits, combined with this target's accessibility this semester, there is likely not enough unique nights in the semester to complete this request. Consider ammending your request, or accept that it may not be able to be fully completed. 



# Example 2
Request to observe a star twice per night, separated by at least 3 hours. Get 25 unique nights at nightly cadence.

In [3]:
Example2 = rsf.Request("HD 219134")
Example2.get_star_info()

# Host star info
Example2.Teff = 5000

#Observation info
Example2.simulcal = False
Example2.nominal_ExpTime = 300
Example2.max_ExpTime = 300

#Cadence info
Example2.n_observations_per_visit = 1
Example2.n_visits_per_night = 2
Example2.n_unique_nights_per_semester = 25
Example2.intra_night_cadence = 3.0
Example2.inter_night_cadence = 1

# run checks and calculations
Example2.computeTimeRequest()
Example2.runChecks()
if Example2.canContinue:
    print("Passed all checks.")
    Example2.isObservable()
    Example2.determineFeasibility()
else:
    print("Did not pass all checks. Make changes, then re-run.")

Checking for parameters still set to None.

Checking for correct data types.

Checking for formatting.

All looks good for this request! 

Passed all checks.
This target is observable for a total of [180] days this semester.
This target rises on day [0] and sets on day [180] of the semester. 

To first order checks, this target's request is feasible. This is not a guarentee that it will actually be completed to 100% of the request.


# Example 3
Request to get a single observation a target, without any cadence requirements. But do require it be taken within a single specified month.

In [4]:
Example3 = rsf.Request("Tau Ceti")
Example3.get_single_shot()

# Host star info
Example3.Teff = 5300

#Observation info
Example3.simulcal = False
Example3.nominal_ExpTime = 10
Example3.max_ExpTime = 10

Example3.startDate = '2024-05-01'
Example3.endDate = '2024-06-01'

# be sure not to include an commas in your note otherwise it messes up the resulting CSV!
Example3.notes = 'Binary companion; observe NE component.'

# run checks and calculations
Example3.computeTimeRequest()
Example3.runChecks()
if Example3.canContinue:
    print("Passed all checks.")
    Example3.isObservable()
    Example3.determineFeasibility()
else:
    print("Did not pass all checks. Make changes, then re-run.")


Checking for parameters still set to None.

Checking for correct data types.

Checking for formatting.

All looks good for this request! 

Passed all checks.
This target is observable for a total of [180] days this semester.
This target rises on day [0] and sets on day [180] of the semester. 

To first order checks, this target's request is feasible. This is not a guarentee that it will actually be completed to 100% of the request.


# Example 4
Submit a request to get an RM measurement. This is a bit trickier because of the time sensativity. Right now, RM and similar time sensative observations are scheduled manually. But we would still like to ask for this information. For these kinds of observations, we will work with you directly to ensure all observations are scheduled correctly.

In [5]:
Example4 = rsf.Request("51 Pegasi")
Example4.get_star_info()

# First set exposure time info; must set this first before running get_RM
Example4.simulcal = True
Example4.nominal_ExpTime = 240
Example4.max_ExpTime = 240
Example4.eventWindow = 5 #time length for the event window (including any pre/post transit observations), units of hours

# Host star info
# Example4.Jmag = 5.5
Example4.Teff = 5700

# Now run the RM command, which auto computes the 
# total number of exposures needed and fills in cadence info appropriately
Example4.get_RM()

# Specify the UT date of the observation
Example4.mustDate = ['2024-03-15']

# run checks and calculations
Example4.computeTimeRequest_RM()
Example4.runChecks()
if Example4.canContinue:
    print("Passed all checks.")
    Example4.isObservable()
    Example4.determineFeasibility()
else:
    print("Did not pass all checks. Make changes, then re-run.")
    

Checking for parameters still set to None.

Checking for correct data types.

Checking for formatting.

All looks good for this request! 

Passed all checks.
This target is observable for a total of [91] days this semester.
This target rises on day [89] and sets on day [180] of the semester. 

To first order checks, this target's request is feasible. This is not a guarentee that it will actually be completed to 100% of the request.


# Additional Targets
This notebook shows only four examples but please add more cells below to build Request objects for each of your targets in your program. 

Only when each target has a Request object describing its desired observational strategy, then move on to build the Program object, as exampled below.

In [6]:
# place all Request Objects into a new Program object
myProgram = rsf.Program([Example1, Example2, Example3, Example4])

# Your 4 letter/number proposal code as supplied at the time of submission of the proposal 
# This is used for easy sorting of targets within programs and tracking overall time accounting
myProgram.code = 'X999'
myProgram.semester = '2024A' # Do not change

path = '/Users/jack/Desktop/'# YOUR SAVE PATH HERE
myProgram.savefile = path + str(myProgram.code) + "_KPFCC_2024A.csv" #suggested filename

# run final time accounting, final checks, then write the csv file
myProgram.totalProgramTime()
myProgram.checkAllRequests()
myProgram.writeFile()

This program requires an allocation of [3.01] nights to be feasible.
Sirius,No Gaia Name,TIC322899250,X999,101.29,-16.72,-546.01,-1223.07,J2000,-1.36,5708,10,10,True,150,3,1,0.0,50,3,51000,14.17,,,,This is the brightest star in the sky... cant miss it!

HD 219134,Gaia_DR2_2009481748875806976,TIC283722336,X999,348.32,57.17,2074.414,294.452,J2000,3.86,5000,300,300,False,50,1,2,3.0,25,1,39000,10.83,,,,

Tau Ceti,Gaia_DR2_2452378776434276992,TIC419015728,X999,26.02,-15.94,-1721.728,854.963,J2000,2.14,5300,10,10,False,1,1,1,0.0,1,0,250,0.07,2024-05-01,2024-06-01,,Binary companion; observe NE component.

51 Pegasi,Gaia_DR2_2835207319109249920,TIC139298196,X999,344.37,20.77,207.328,61.164,J2000,4.66,5700,240,240,True,64,64,1,0.0,1,0,18000,5,,,['2024-03-15'],



# Last instruction
Please email your csv to jblubin@astro.ucla.edu

Please feel encouraged to also describe your experience running this notebook. Is it easy or difficult? Is it intuitive or confusing? Is there additional information about your program and your requests that you feel is not captured within this framework? Any and all comments are welcome. We will be reaching out with a more formal survey asking for feedback on your use of this notebook. 

In [7]:
readin = pd.read_csv(myProgram.savefile)
readin

,Name,Gaia_ID,TIC_ID,Program_Code,RA,Dec,pmRA,pmDec,Epoch,Jmag,...,N_Visits_per_Night,Intra_Night_Cadence,N_Unique_Nights,Inter_Night_Cadence,Total_Time_for_Target_Seconds,Total_Time_for_Target_Hours,Start_Date,End_Date,Must_Dates,Notes
0,Sirius,No Gaia Name,TIC322899250,X999,101.29,-16.72,-546.010,-1223.070,J2000,-1.36,...,1,0.0,50,3,51000,14.17,NaN,NaN,NaN,This is the brightest star in the sky... cant ...
1,HD 219134,Gaia_DR2_2009481748875806976,TIC283722336,X999,348.32,57.17,2074.414,294.452,J2000,3.86,...,2,3.0,25,1,39000,10.83,NaN,NaN,NaN,NaN
2,Tau Ceti,Gaia_DR2_2452378776434276992,TIC419015728,X999,26.02,-15.94,-1721.728,854.963,J2000,2.14,...,1,0.0,1,0,250,0.07,2024-05-01,2024-06-01,NaN,Binary companion; observe NE component.
3,51 Pegasi,Gaia_DR2_2835207319109249920,TIC139298196,X999,344.37,20.77,207.328,61.164,J2000,4.66,...,1,0.0,1,0,18000,5.00,NaN,NaN,['2024-03-15'],NaN
